In [36]:
import pinocchio as pin
import numpy as np
import sys
import time
import os
from os.path import dirname, join, abspath
from pinocchio.utils import *
import crocoddyl

from pinocchio.visualize import MeshcatVisualizer

In [124]:
# Load the URDF model.

#pinocchio_model_dir = dirname(dirname(str(abspath(__file__))))
pinocchio_model_dir = os.path.dirname(sys.executable)

model_path = join(pinocchio_model_dir, "example-robot-data/robots")
mesh_dir = pinocchio_model_dir


urdf_model_path='/Users/dkuzovlev/Documents/Robots/pinocchio-tutorial-master/dp_1_1.urdf'

model, collision_model, visual_model = pin.buildModelsFromUrdf(
    urdf_model_path, mesh_dir#, pin.JointModelFreeFlyer()
)
data = model.createData()

#urdf_model2_path='/Users/dkuzovlev/Documents/Robots/pinocchio-tutorial-master/dp_3_2.urdf'
#model2, collision_model2, visual_model2 = pin.buildModelsFromUrdf(urdf_model2_path, mesh_dir#, pin.JointModelFreeFlyer())
#data2 = model2.createData()


In [125]:
#Meshcat initialisation (dependency: pip install meshcat)

try:
    viz = MeshcatVisualizer(model, collision_model, visual_model)
except ImportError as err:
    print(
            "Error while initializing the viewer. It seems you should install Python meshcat"
        )
    print(err)
    sys.exit(0)

viz.initViewer(open=True)



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7029/static/


In [129]:
#display random configuration
q0 = pin.neutral(model)
q1 = pin.randomConfiguration(model)
v1 = np.random.randn(model.nv)

# Load the robot in the viewer.
viz.loadViewerModel()
# Display a robot configuration.
viz.displayVisuals(True)
viz.display(q1)

com = pin.centerOfMass(model, data, q1)

print("Center of Mass:", com)


Center of Mass: [0.35441148 0.         0.47162584]


In [131]:

# Motion

q1 = pin.randomConfiguration(model)
v1 = np.random.randn(model.nv)
v1=v1*2
data = viz.data
             


model.gravity.linear = np.array([0, 0, -9.81])
dt = 0.01
 
 
def sim_loop():
    tau0 = np.zeros(model.nv) #Joint torques
    qs = [q1]
    vs = [v1] #Joint velocities
    nsteps = 10000
    for i in range(nsteps):
        q = qs[i]
        v = vs[i] 
        a1 = pin.aba(model, data, q, v, tau0) #joint acceleration (a).
        vnext = v + dt * a1
        qnext = pin.integrate(model, q, dt * vnext) # The integrate method takes the current joint configuration (q) and a small increment in the joint velocities (dq) and computes the new configuration (q'). This method allows you to move from one configuration to another based on the robot’s kinematics.
        qs.append(qnext)
        vs.append(vnext)
        viz.display(qnext)
        time.sleep(0.01)
        #viz.drawFrameVelocities(frame_id=frame_id)
    return qs, vs
qs, vs = sim_loop() 

KeyboardInterrupt: 